In [5]:
%matplotlib inline
import pandas as pd
from collections import Counter
import json
import ast
from pandas.io.json import json_normalize
import requests
import re
from datetime import datetime
import sqlite3
from  sqlalchemy import create_engine


In [6]:
DB_NAME = 'ft_comments.db'
disk_engine = create_engine('sqlite:///' + DB_NAME)

# Process authors

In [7]:
def get_authors_list(x):
    
    try:
        authors = [(k,v[u'displayName']) for k,v in x.iteritems()]
        return authors
    except Exception:
        return ""

def create_authors_df(df):
    df['authors_list'] = df.authors.apply(get_authors_list)
    b_flat = pd.DataFrame([[i, x] for i, y in df.authors_list.apply(list).iteritems() 
                           for x in y], columns=list('IB'))
    df_authors= b_flat.B.apply(pd.Series).drop_duplicates(0)
    df_authors.columns=['authorId','authorName']
    df_authors.drop_duplicates(inplace=True)
    #save_authors_to_db(df_authors)
    

    return df_authors

    
def save_authors_to_db(df):
    try:
        df_authors_db = pd.read_sql_table('authors',con=disk_engine)
        df_new_authors = df[~df.authorId.isin(df_authors_db.authorId.tolist())]
        df_new_authors.to_sql('authors',con=disk_engine,if_exists='append')
    
    except Exception as e:
        
        df.to_sql('authors',con=disk_engine)
        
    

In [8]:
def process_authors(df):
    
    df['authors'] = df.comments.apply(pd.Series)['authors']
    create_authors_df(df)
    print ("Finished processing authors")

## Process comments

In [23]:
def to_dict(x):
    try:
        d = json.dumps(ast.literal_eval(x))
        return json.loads(d)
    except Exception:
        pass
    
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

def extract_liked_by(d):
    likedBy = []
    if 'childContent' in d.keys():
        for child in d['childContent']:
            if child['type']==1:
                likedBy.append(child['content']['authorId'])
    return likedBy
                

def extract_content(d,FT_ID,title):
    try:
        authorId = d['content']['authorId']
    except:
        authorId = ""
        
    try:
        parentId = d['content']['parentId']
    except:
        parentId = ""
    
    try:
        commentId = d['content']['id']
    except:
        commentId = ""
        
    try:
        ancestorId = d['content']['ancestorId']
    except:
        ancestorId = ""
    
    try:
        messagetag = d['content']['annotations']['messagetag']
    except:
        messagetag = ""
    try:
        bodyHtml = cleanhtml(d['content']['bodyHtml'])
    except:
        bodyHtml = ""
    try:
        createdAt = d['content']['createdAt']
    except:
        createdAt = ""
    try:
        updatedAt=d['content']['updatedAt']
    except:
        updatedAt = ""
    try:
        likedBy = extract_liked_by(d)
    except :
        likedBy = ""
        
    return ((FT_ID,title,commentId,ancestorId,parentId,authorId,messagetag,bodyHtml,createdAt,updatedAt,likedBy))
    

def flatten_dic_content(dico,L,FT_ID,title):
    L.append(extract_content(dico,FT_ID,title))
    if 'childContent' in dico.keys():
        for c in dico['childContent']:
            if c['type']==0:
                flatten_dic_content(c,L,FT_ID,title)
            

def flatten_comments(content,L):
    
    try:
        
        if isinstance(content.comments['content'],list):
            
            for c in content.comments['content']:
                flatten_dic_content(c,L,content.FT_ID,content.title)
    except Exception as e :
        pass


        
def save_comments_to_db(df):
    try:
        df_comments_db = pd.read_sql_table('comments',con=disk_engine)
        df_new_comments = df[~df.comment_id.isin(df_comments_db.comment_id.tolist())]
        df_new_comments.to_sql('comments',con=disk_engine,if_exists='append', flavor='sqlite')
    
    except Exception as e:
        
        print ('Cannot add to existing table, try creating new one')
        
        df.to_sql('comments',con=disk_engine,if_exists='replace', flavor='sqlite')

In [24]:
def process_comments(df):
    df.comments = df.comments.apply(to_dict)
    df=df[[u'FT_ID', u'LIVEFYRE_ID', u'title', u'comments']]
    
    COMMENTS = []
    
    for index, row in df.iterrows():
        flatten_comments(row,COMMENTS)
        
    df_comment = pd.DataFrame(COMMENTS,columns=['ft_id','title','comment_id','ancestor_id',
                                                'parent_id','author_id','messagetag','comment',
                                                'created_at','updated_at','liked_by'])
   
    df_comment['liked_by'] = df_comment.liked_by.astype(str)
    df_comment['messagetag'] = df_comment.messagetag.astype(str)
    
    #df_comment['comment'] = 1 #df_comment['comment'].str.encode('utf-8')
    save_comments_to_db(df_comment)
    print( "Finished processing comments")
    

## Process in chunks

In [11]:
def process_chunk():
    
    for chunk in pd.read_csv('df_all_comments.csv', chunksize=1000,encoding='utf-8'):
          
        #process_authors(chunk)
        print chunk.shape
        
    
        process_comments(chunk)
        
    

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(chunk.shape)? (<ipython-input-11-95435935d42d>, line 6)

In [8]:
#process_chunk()

In [9]:
df_comment= pd.read_sql_table('comments',con=disk_engine)
df_authors = pd.read_sql_table('authors',con=disk_engine)
df_comment = pd.merge(df_comment,df_authors,left_on='author_id',right_on='authorId')



In [10]:
df_comment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 258307 entries, 0 to 258306
Data columns (total 15 columns):
index_x        258307 non-null int64
ft_id          258307 non-null object
title          258307 non-null object
comment_id     258307 non-null object
ancestor_id    258307 non-null object
parent_id      258307 non-null object
author_id      258307 non-null object
messagetag     258307 non-null object
comment        258307 non-null object
created_at     258307 non-null int64
updated_at     258307 non-null object
liked_by       258307 non-null object
index_y        258307 non-null int64
authorId       258307 non-null object
authorName     258307 non-null object
dtypes: int64(3), object(12)
memory usage: 31.5+ MB


In [11]:
df_comment.groupby('authorName').count().sort_values(by='ft_id',ascending=False).head(20)

,index_x,ft_id,title,comment_id,ancestor_id,parent_id,author_id,messagetag,comment,created_at,updated_at,liked_by,index_y,authorId
authorName,,,,,,,,,,,,,,
RiskAdjustedReturn,2210,2210,2210,2210,2210,2210,2210,2210,2210,2210,2210,2210,2210,2210
Njegos,1705,1705,1705,1705,1705,1705,1705,1705,1705,1705,1705,1705,1705,1705
genauer,1578,1578,1578,1578,1578,1578,1578,1578,1578,1578,1578,1578,1578,1578
maljoffre,1522,1522,1522,1522,1522,1522,1522,1522,1522,1522,1522,1522,1522,1522
New russian troll (ex christopher c),1392,1392,1392,1392,1392,1392,1392,1392,1392,1392,1392,1392,1392,1392
Judyw,1390,1390,1390,1390,1390,1390,1390,1390,1390,1390,1390,1390,1390,1390
Good European,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278
Ric,1183,1183,1183,1183,1183,1183,1183,1183,1183,1183,1183,1183,1183,1183
Coase Theorem,1137,1137,1137,1137,1137,1137,1137,1137,1137,1137,1137,1137,1137,1137


In [12]:
top_authors = [u'RiskAdjustedReturn', u'genauer', u'Njegos', u'Judyw', u'maljoffre',
       u'New russian troll (ex christopher c)', u'Ric', u'Good European',
       u'Footsteps', u'Paul A. Myers', u'Coase Theorem', u'RiskManager',
       u'onomasticator', u'Sound of the Suburbs', u'Sus Scrofa', u'Ealing',
       u'Alfred Nassim', u'Padme Hum', u'RandomCommenter7',
       u'Brexit means Leave!']

In [38]:
df_comment['datetime']=df_comment.created_at.apply(datetime.fromtimestamp)
df_comment['datedatetime']=df_comment.datetime.dt.date

In [14]:
df_comment['time']= df_comment.datetime.dt.hour
df_comment['datetime_moscow']= df_comment.datetime +  pd.DateOffset(hours=15)
df_comment['time_moscow']= df_comment.datetime_moscow.dt.hour

In [39]:
pd.DataFrame(df_comment[df_comment.authorName==u'The Zeus']['date'].value_counts()).head()

,date
2015-12-08,7
2016-04-22,7
2017-02-25,6
2016-09-16,6
2017-05-23,6


In [45]:
df_comment[df_comment.authorName=='The Zeus'].groupby('date').count()

,index_x,ft_id,title,comment_id,ancestor_id,parent_id,author_id,messagetag,comment,created_at,updated_at,liked_by,index_y,authorId,authorName,datetime,time,datetime_moscow,time_moscow,likes_count
date,,,,,,,,,,,,,,,,,,,,
2014-06-20,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2014-06-26,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2014-07-07,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2014-07-08,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2014-07-18,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2014-07-23,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2014-08-08,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2014-08-26,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2014-09-02,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [17]:
df_comment['likes_count']=df_comment.liked_by.apply(lambda x:len(x))

In [18]:
df_comment.sort_values(by='likes_count',ascending=False)
df_comment = df_comment[df_comment.likes_count>10]

In [19]:
df_comment.liked_by = df_comment.liked_by.apply(ast.literal_eval)
s = df_comment.apply(lambda x: pd.Series(x['liked_by']),axis=1).stack().reset_index(level=1, drop=True)
s.name='liker'

In [24]:
df_likes = df_comment.drop('liked_by', axis=1)[['author_id']].join(s).dropna().reset_index()

In [25]:
df_likes.head()

,index,author_id,liker
0,1,7148686@ft.fyre.co,2000177198@ft.fyre.co
1,2,7148686@ft.fyre.co,10924558@ft.fyre.co
2,2,7148686@ft.fyre.co,12484241@ft.fyre.co
3,2,7148686@ft.fyre.co,12377500@ft.fyre.co
4,2,7148686@ft.fyre.co,6823142@ft.fyre.co


In [27]:
df_likes = pd.merge(df_likes,df_authors,left_on='author_id',right_on='authorId')
df_likes.drop('authorId',inplace=True,axis=1)
df_likes.head()

,index_x,author_id,liker,index_y,authorName
0,1,7148686@ft.fyre.co,2000177198@ft.fyre.co,173,Good European
1,2,7148686@ft.fyre.co,10924558@ft.fyre.co,173,Good European
2,2,7148686@ft.fyre.co,12484241@ft.fyre.co,173,Good European
3,2,7148686@ft.fyre.co,12377500@ft.fyre.co,173,Good European
4,2,7148686@ft.fyre.co,6823142@ft.fyre.co,173,Good European


In [28]:
df_likes = pd.merge(df_likes,df_authors,left_on='liker',right_on='authorId')

In [29]:
df_likes.head()

,index_x,author_id,liker,index_y,authorName_x,index,authorId,authorName_y
0,1,7148686@ft.fyre.co,2000177198@ft.fyre.co,173,Good European,176,2000177198@ft.fyre.co,Mik
1,372,7148686@ft.fyre.co,2000177198@ft.fyre.co,173,Good European,176,2000177198@ft.fyre.co,Mik
2,374,7148686@ft.fyre.co,2000177198@ft.fyre.co,173,Good European,176,2000177198@ft.fyre.co,Mik
3,473,7148686@ft.fyre.co,2000177198@ft.fyre.co,173,Good European,176,2000177198@ft.fyre.co,Mik
4,1484,10906029@ft.fyre.co,2000177198@ft.fyre.co,740,GrahamHanson,176,2000177198@ft.fyre.co,Mik


In [30]:
df_likes_grouped = df_likes.groupby(['authorName_x','authorName_y']).count().reset_index()#.unstack('authorName_y')
df_likes_grouped = df_likes_grouped.fillna(0)
df_likes_grouped.head()

,authorName_x,authorName_y,index_x,author_id,liker,index_y,index,authorId
0,!,.,2,2,2,2,2,2
1,!,123,1,1,1,1,1,1
2,!,3121791,1,1,1,1,1,1
3,!,A Guy,1,1,1,1,1,1
4,!,A lawyer writes,1,1,1,1,1,1


In [31]:
df_likes_grouped['Total'] = df_likes_grouped.sum(axis=1)
df_likes_grouped = df_likes_grouped.loc[:, (df_likes_grouped != 0).any(axis=0)]

In [32]:
df_likes_grouped.sort_values('Total',ascending=False)

,authorName_x,authorName_y,index_x,author_id,liker,index_y,index,authorId,Total
211710,Njegos,Francheska (Maya) Smith-Johnson,335,335,335,335,335,335,2010
312122,TylerDurden,Coase Theorem,271,271,271,271,271,271,1626
59966,Coase Theorem,TylerDurden,229,229,229,229,229,229,1374
369991,maljoffre,Njegos,216,216,216,216,216,216,1296
369737,maljoffre,Francheska (Maya) Smith-Johnson,192,192,192,192,192,192,1152
83607,Ealing,Francheska (Maya) Smith-Johnson,152,152,152,152,152,152,912
207144,New russian troll (ex christopher c),Francheska (Maya) Smith-Johnson,137,137,137,137,137,137,822
207524,New russian troll (ex christopher c),Njegos,136,136,136,136,136,136,816
255944,Ric,Njegos,134,134,134,134,134,134,804
62929,Contrarian Investor,MaZG,116,116,116,116,116,116,696
